## 5. 设计并训练NN算法对图片进行分类。

In [1]:
import numpy as np
import struct
import matplotlib.pyplot as plt
import random
import pickle

In [2]:
train_num = 1000

In [3]:
class Data:
    def __init__(self):

        self.K = 10
        self.N = 60000
        self.M = 10000
        self.BATCHSIZE = 2000
        self.reg_factor = 1e-3
        self.stepsize = 1e-2
        self.train_img_list = np.zeros((self.N, 28 * 28))
        self.train_label_list = np.zeros((self.N, 1))

        self.test_img_list = np.zeros((self.M, 28 * 28))
        self.test_label_list = np.zeros((self.M, 1))

        self.loss_list = []
        self.init_network()

        self.read_train_images('/home/vbuo/m-L-0/data/fashion/train-images-idx3-ubyte')
        self.read_train_labels('/home/vbuo/m-L-0/data/fashion/train-labels-idx1-ubyte')

        self.train_data = np.append(
            self.train_img_list, self.train_label_list, axis=1)

        self.read_test_images('/home/vbuo/m-L-0/data/fashion/t10k-images-idx3-ubyte')
        self.read_test_labels('/home/vbuo/m-L-0/data/fashion/t10k-labels-idx1-ubyte')

    def predict(self):
        hidden_layer1 = np.maximum(
            0, np.matmul(self.test_img_list, self.W1) + self.b1)

        hidden_layer2 = np.maximum(0,
                                   np.matmul(hidden_layer1, self.W2) + self.b2)

        scores = np.maximum(0, np.matmul(hidden_layer2, self.W3) + self.b3)

        prediction = np.argmax(scores, axis=1)
        prediction = np.reshape(prediction, (10000, 1))
        print(prediction.shape)
        print(self.test_label_list.shape)
        accuracy = np.mean(prediction == self.test_label_list)
        print('The accuracy is:  ', accuracy)
        return

    def train(self):

        for i in range(train_num):
            np.random.shuffle(self.train_data)
            img_list = self.train_data[:self.BATCHSIZE, :-1]
            label_list = self.train_data[:self.BATCHSIZE, -1:]
            print("Train Time: ", i)
            self.train_network(img_list, label_list)

    def train_network(self, img_batch_list, label_batch_list):

        # calculate softmax
        train_example_num = img_batch_list.shape[0]
        hidden_layer1 = np.maximum(
            0, np.matmul(img_batch_list, self.W1) + self.b1)

        hidden_layer2 = np.maximum(0,
                                   np.matmul(hidden_layer1, self.W2) + self.b2)

        scores = np.maximum(0, np.matmul(hidden_layer2, self.W3) + self.b3)

        scores_e = np.exp(scores)
        scores_e_sum = np.sum(scores_e, axis=1, keepdims=True)

        probs = scores_e / scores_e_sum

        loss_list_tmp = np.zeros((train_example_num, 1))
        for i in range(train_example_num):
            loss_list_tmp[i] = scores_e[i][int(label_batch_list[
                i])] / scores_e_sum[i]
        loss_list = -np.log(loss_list_tmp)

        loss = np.mean(loss_list, axis=0)[0] + 0.5 * self.reg_factor * np.sum(self.W1 * self.W1) + 0.5 * self.reg_factor * np.sum(self.W2 * self.W2) + 0.5 * self.reg_factor * np.sum(self.W3 * self.W3)

        self.loss_list.append(loss)
        print(loss, " ", len(self.loss_list))
        # backpropagation

        dscore = np.zeros((train_example_num, self.K))
        for i in range(train_example_num):
            dscore[i][:] = probs[i][:]
            dscore[i][int(label_batch_list[i])] -= 1

        dscore /= train_example_num

        dW3 = np.dot(hidden_layer2.T, dscore)
        db3 = np.sum(dscore, axis=0, keepdims=True)

        dh2 = np.dot(dscore, self.W3.T)
        dh2[hidden_layer2 <= 0] = 0

        dW2 = np.dot(hidden_layer1.T, dh2)
        db2 = np.sum(dh2, axis=0, keepdims=True)

        dh1 = np.dot(dh2, self.W2.T)
        dh1[hidden_layer1 <= 0] = 0

        dW1 = np.dot(img_batch_list.T, dh1)
        db1 = np.sum(dh1, axis=0, keepdims=True)

        dW3 += self.reg_factor * self.W3
        dW2 += self.reg_factor * self.W2
        dW1 += self.reg_factor * self.W1

        self.W3 += -self.stepsize * dW3
        self.W2 += -self.stepsize * dW2
        self.W1 += -self.stepsize * dW1

        self.b3 += -self.stepsize * db3
        self.b2 += -self.stepsize * db2
        self.b1 += -self.stepsize * db1

        return

    def init_network(self):
        self.W1 = 0.01 * np.random.randn(28 * 28, 100)
        self.b1 = 0.01 * np.random.randn(1, 100)

        self.W2 = 0.01 * np.random.randn(100, 20)
        self.b2 = 0.01 * np.random.randn(1, 20)

        self.W3 = 0.01 * np.random.randn(20, self.K)
        self.b3 = 0.01 * np.random.randn(1, self.K)

    def read_train_images(self, filename):
        binfile = open(filename, 'rb')
        buf = binfile.read()
        index = 0
        magic, self.train_img_num, self.numRows, self.numColums = struct.unpack_from(
            '>IIII', buf, index)
        print(magic, ' ', self.train_img_num, ' ', self.numRows, ' ', self.numColums)
        index += struct.calcsize('>IIII')
        for i in range(self.train_img_num):
            im = struct.unpack_from('>784B', buf, index)
            index += struct.calcsize('>784B')
            im = np.array(im)
            im = im.reshape(1, 28 * 28)
            self.train_img_list[i, :] = im

            # plt.imshow(im, cmap='binary')  # 黑白显示
            # plt.show()

    def read_train_labels(self, filename):
        binfile = open(filename, 'rb')
        index = 0
        buf = binfile.read()
        binfile.close()

        magic, self.train_label_num = struct.unpack_from('>II', buf, index)
        index += struct.calcsize('>II')

        for i in range(self.train_label_num):
            # for x in xrange(2000):
            label_item = int(struct.unpack_from('>B', buf, index)[0])
            self.train_label_list[i, :] = label_item
            index += struct.calcsize('>B')

    def read_test_images(self, filename):
        binfile = open(filename, 'rb')
        buf = binfile.read()
        index = 0
        magic, self.test_img_num, self.numRows, self.numColums = struct.unpack_from(
            '>IIII', buf, index)
        print(magic, ' ', self.test_img_num, ' ', self.numRows, ' ', self.numColums)
        index += struct.calcsize('>IIII')
        for i in range(self.test_img_num):
            im = struct.unpack_from('>784B', buf, index)
            index += struct.calcsize('>784B')
            im = np.array(im)
            im = im.reshape(1, 28 * 28)
            self.test_img_list[i, :] = im

    def read_test_labels(self, filename):
        binfile = open(filename, 'rb')
        index = 0
        buf = binfile.read()
        binfile.close()

        magic, self.test_label_num = struct.unpack_from('>II', buf, index)
        index += struct.calcsize('>II')

        for i in range(self.test_label_num):
            # for x in xrange(2000):
            label_item = int(struct.unpack_from('>B', buf, index)[0])
            self.test_label_list[i, :] = label_item
            index += struct.calcsize('>B')

In [4]:
def main():
    data = Data()
    data.train()
    data.predict()

In [5]:
if __name__ == '__main__':
    main()

2051   60000   28   28
2051   10000   28   28
Train Time:  0
2.3079527399   1
Train Time:  1
2.29601004318   2
Train Time:  2
2.2831806986   3
Train Time:  3
2.26417928583   4
Train Time:  4
2.24362168452   5
Train Time:  5
2.21251592757   6
Train Time:  6
2.17073878084   7
Train Time:  7
2.10505962583   8
Train Time:  8
2.02275398547   9
Train Time:  9
1.92458365245   10
Train Time:  10
1.89513907489   11
Train Time:  11
2.15332176217   12
Train Time:  12
2.33860186254   13
Train Time:  13
2.33735798513   14
Train Time:  14
2.11249767922   15
Train Time:  15
2.01620007816   16
Train Time:  16
1.84615027561   17
Train Time:  17
1.65538873045   18
Train Time:  18
1.48008808377   19
Train Time:  19
1.44206232355   20
Train Time:  20
1.80407112202   21
Train Time:  21
2.43564852992   22
Train Time:  22
2.04599091518   23
Train Time:  23
1.89544269538   24
Train Time:  24
1.78281862316   25
Train Time:  25
1.50491436411   26
Train Time:  26
1.41516992668   27
Train Time:  27
1.33310614183 

Train Time:  223
0.533118156742   224
Train Time:  224
0.524715047945   225
Train Time:  225
0.530352412571   226
Train Time:  226
0.571285026308   227
Train Time:  227
0.627544578188   228
Train Time:  228
0.684955845397   229
Train Time:  229
0.626110461309   230
Train Time:  230
0.569820121228   231
Train Time:  231
0.505135121649   232
Train Time:  232
0.547078701757   233
Train Time:  233
0.54254572132   234
Train Time:  234
0.559957184124   235
Train Time:  235
0.580693793828   236
Train Time:  236
0.57358301252   237
Train Time:  237
0.489479991751   238
Train Time:  238
0.494767769322   239
Train Time:  239
0.494731690293   240
Train Time:  240
0.50293563401   241
Train Time:  241
0.555698347839   242
Train Time:  242
0.620178063845   243
Train Time:  243
0.581005630524   244
Train Time:  244
0.54072059025   245
Train Time:  245
0.491882245443   246
Train Time:  246
0.538893591686   247
Train Time:  247
0.537288914334   248
Train Time:  248
0.581044873775   249
Train Time:  249

Train Time:  440
0.439062732087   441
Train Time:  441
0.477114332207   442
Train Time:  442
0.46789661408   443
Train Time:  443
0.437179490809   444
Train Time:  444
0.471373502105   445
Train Time:  445
0.501513340199   446
Train Time:  446
0.486356156467   447
Train Time:  447
0.543605319331   448
Train Time:  448
0.41962106051   449
Train Time:  449
0.400544272047   450
Train Time:  450
0.424362117068   451
Train Time:  451
0.409912785521   452
Train Time:  452
0.369486568467   453
Train Time:  453
0.442391023041   454
Train Time:  454
0.443518405764   455
Train Time:  455
0.374444786349   456
Train Time:  456
0.410455976378   457
Train Time:  457
0.43063985304   458
Train Time:  458
0.374300175043   459
Train Time:  459
0.464841530169   460
Train Time:  460
0.464059118271   461
Train Time:  461
0.49893403203   462
Train Time:  462
0.410257554383   463
Train Time:  463
0.397743493776   464
Train Time:  464
0.401177258758   465
Train Time:  465
0.382713068711   466
Train Time:  466

Train Time:  657
0.369907368748   658
Train Time:  658
0.396793185099   659
Train Time:  659
0.422232139509   660
Train Time:  660
0.410780193777   661
Train Time:  661
0.394282502496   662
Train Time:  662
0.372179459315   663
Train Time:  663
0.407355393804   664
Train Time:  664
0.38779240902   665
Train Time:  665
0.396591854756   666
Train Time:  666
0.396162857467   667
Train Time:  667
0.40595713115   668
Train Time:  668
0.38748614076   669
Train Time:  669
0.41582006616   670
Train Time:  670
0.406457389704   671
Train Time:  671
0.376128539897   672
Train Time:  672
0.387845816063   673
Train Time:  673
0.363462336409   674
Train Time:  674
0.387728135616   675
Train Time:  675
0.350433784721   676
Train Time:  676
0.40111939921   677
Train Time:  677
0.462093679585   678
Train Time:  678
0.425047173922   679
Train Time:  679
0.39315189661   680
Train Time:  680
0.393863068096   681
Train Time:  681
0.431780823426   682
Train Time:  682
0.398835095091   683
Train Time:  683
0

Train Time:  874
0.362400323917   875
Train Time:  875
0.347683280176   876
Train Time:  876
0.355348651095   877
Train Time:  877
0.359196495591   878
Train Time:  878
0.335122448594   879
Train Time:  879
0.335362277881   880
Train Time:  880
0.358176027152   881
Train Time:  881
0.384774227858   882
Train Time:  882
0.385038837758   883
Train Time:  883
0.428132536224   884
Train Time:  884
0.403329254692   885
Train Time:  885
0.381121608846   886
Train Time:  886
0.354629719527   887
Train Time:  887
0.38919947395   888
Train Time:  888
0.389302323697   889
Train Time:  889
0.372480747716   890
Train Time:  890
0.344670771158   891
Train Time:  891
0.391991686089   892
Train Time:  892
0.373315062803   893
Train Time:  893
0.376715905487   894
Train Time:  894
0.356335829235   895
Train Time:  895
0.347462560747   896
Train Time:  896
0.364757905623   897
Train Time:  897
0.344878419267   898
Train Time:  898
0.353518875786   899
Train Time:  899
0.352687334733   900
Train Time:  